# Uni-Fold Folding

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from prtm.models.unifold.modeling import UniFoldForFolding
from prtm.visual import view_superimposed_structures

## Fold Monomer

Folding of monomers with `Uni-Fold` and `AlphaFold` trained models.

In [ ]:
monomer_sequence = (
    "LILNLRGGAFVSNTQITMADKQKKFINEIQEGDLVRSYSITDETFQQNAVTSIV"
    "KHEADQLCQINFGKQHVVCTVNHRFYDPESKLWKSVCPHPGSGISFLKKYDYLLS"
    "EEGEKLQITEIKTFTTKQPVFIYHIQVENNHNFFANGVLAHAMQVSI"
)
monomer_sequence_dict = {"A": monomer_sequence}

In [ ]:
# Uni-Fold trained model
uf_folder = UniFoldForFolding(model_name="model_2_ft", use_templates=True, random_seed=0)
# AlphaFold trained model converted to PyTorch for Uni-Fold
af_folder = UniFoldForFolding(model_name="model_1_af2", use_templates=True, random_seed=0)

In [ ]:
uf_monomer_structure, uf_aux_output = uf_folder(monomer_sequence_dict, max_recycling_iters=3, num_ensembles=2)
af_monomer_structure, af_aux_output = af_folder(monomer_sequence_dict, max_recycling_iters=3, num_ensembles=2)

In [ ]:
view_superimposed_structures(uf_monomer_structure, af_monomer_structure, color1="green")

## Fold Multimer

Folding of multimers with `Uni-Fold` and `AlphaFold` trained models.

In [ ]:
complex_sequence_a = (
    "TTPLVHVASVEKGRSYEDFQKVYNAIALKLREDDEYDNYIGYGPVLVRLAWHTSGTW"
    "DKHDNTGGSYGGTYRFKKEFNDPSNAGLQNGFKFLEPIHKEFPWISSGDLFSLGGVTA"
    "VQEMQGPKIPWRCGRVDTPEDTTPDNGRLPDADKDADYVRTFFQRLNMNDREVVALMGAH"
    "ALGKTHLKNSGYEGPWGAANNVFTNEFYLNLLNEDWKLEKNDANNEQWDSKSGYMMLPTDY"
    "SLIQDPKYLSIVKEYANDQDKFFKDFSKAFEKLLENGITFPKDAPSPFIFKTLEEQGL"
)
complex_sequence_b = (
    "TEFKAGSAKKGATLFKTRCLQCHTVEKGGPHKVGPNLHGIFGRHSGQAEGYSYTDA"
    "NIKKNVLWDENNMSEYLTNPKKYIPGTKMAIGGLKKEKDRNDLITYLKKACE"
)
complex_sequence_dict = {"A": complex_sequence_a, "B": complex_sequence_b}

In [ ]:
# Uni-Fold Multimer trained model
uf_folder = UniFoldForFolding(model_name="multimer_ft", use_templates=True, random_seed=0)
# AlphaFold Multimer model converted to PyTorch for Uni-Fold
af_folder = UniFoldForFolding(model_name="multimer_4_af2_v3", use_templates=True, random_seed=0)

In [ ]:
uf_complex_structure, uf_comp_aux_output = uf_folder(
    complex_sequence_dict, max_recycling_iters=3, num_ensembles=2
)
af_complex_structure, af_compp_aux_output = af_folder(
    complex_sequence_dict, max_recycling_iters=3, num_ensembles=2
)

In [ ]:
view_superimposed_structures(
    uf_complex_structure.get_chain("A"), af_complex_structure.get_chain("A"), color1="green"
)

In [ ]:
view_superimposed_structures(
    uf_complex_structure.get_chain("B"), af_complex_structure.get_chain("B"), color1="green"
)

## Fold Symmetric

Symmetric folding with `UF-Symmetry`.

In [ ]:
symmetric_sequence = (
    "PPYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKASCLYGQLPKFQDGD"
    "LTLYQSNTILRHLGRTLGLYGKDQQEAALVDMVNDGVEDLRCKYISLIYTNYEAGKDDYV"
    "KALPGQLKPFETLLSQNQGGKTFIVGDQISFADYNLLDLLLIHEVLAPGCLDAFPLLSAY"
    "VGRLSARPKLKAFLASPEYVNLPINGNGKQ"
)
symmetric_sequence_dict = {"A": symmetric_sequence}

In [ ]:
sym_folder = UniFoldForFolding(
    model_name="uf_symmetry", use_templates=True, random_seed=0, symmetry_group="C2"
)

In [ ]:
sym_structure, sym_aux_output = sym_folder(
    symmetric_sequence_dict, max_recycling_iters=3, num_ensembles=2
)

In [ ]:
sym_structure.show()